# State Space Class

Let's consider a time-invariant discret time state space representation as follows.

$$
\begin{align}
    \mathbf{x}_{k+1}&=\mathbf{Ax}_{k}+\mathbf{Bu}_{k} \\
    \mathbf{y}_{k}&=\mathbf{Cx}_{k}+\mathbf{Du}_{k}
\end{align}
$$

We may use a class to implement a state space equation.